# 正規化層

入力されたデータを正規化して返す層。

入力されたデータが特定の統計量（平均・分散）に従うように正規化される。  
統計量は学習可能なパラメータである。

平均0, 分散1への正規化+シフトとも見られる。

統計量を求める軸によって色々な種類がある。
- バッチ正規化
- 層正規化
- インスタンス正規化

In [1]:
import torch
from torch import nn


---

## バッチ正規化

*Batch Normalization*

バッチ方向に正規化を行う。  
学習時と推論時で挙動が変わる。

### 学習

学習時の挙動を見てみよう。  
まず適当なデータを用意する。

In [2]:
batch_size = 2
d = 5
x = torch.randn(batch_size, d)
x

tensor([[-0.9997,  0.1207, -1.0132, -0.0346, -1.1761],
        [ 1.2045, -0.3699,  1.4332,  0.5508, -0.9444]])

バッチ正規化層を作る。pytorchを使う。

In [3]:
norm = nn.BatchNorm1d(d)
norm.train();

データを突っ込んでみよう。

In [4]:
y = norm(x)
y

tensor([[-1.0000,  0.9999, -1.0000, -0.9999, -0.9996],
        [ 1.0000, -0.9999,  1.0000,  0.9999,  0.9996]],
       grad_fn=<NativeBatchNormBackward0>)

平均0, 分散1に正規化された。  
これはpytorchが設定した初期値であって、学習によって変化する。

In [5]:
for params in norm.parameters():
    print(params)

Parameter containing:
tensor([1., 1., 1., 1., 1.], requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0.], requires_grad=True)


学習時はミニバッチ前提なので、例えばバッチサイズ1のデータを入れるとエラーが出る。

In [6]:
x = torch.randn(1, d)
try:
    y = norm(x)
except Exception as e:
    print(e)

Expected more than 1 value per channel when training, got input size torch.Size([1, 5])


### 推論

推論時は挙動が変わる。

学習時と同じようにしてしまうと、バッチ内のデータによって結果が変わってしまう。サンプリング方法によって推論結果が変わるのはマズイ。  
後は、バッチサイズ1の時も困る。

推論時は、学習データ全体の統計量で正規化する。  
学習時に、特徴量ごとの統計量を記録しておく。

みてみよう。

In [7]:
batch_size = 2
d = 5
x = torch.randn(batch_size, d)
x

tensor([[-1.2630, -1.9636, -0.6218,  1.5072,  0.8681],
        [ 0.6868, -1.2388, -0.1524, -2.1604,  0.6878]])

In [8]:
norm = nn.BatchNorm1d(d)
norm.eval();

In [9]:
y = norm(x)
y

tensor([[-1.2630, -1.9635, -0.6218,  1.5072,  0.8681],
        [ 0.6868, -1.2388, -0.1524, -2.1604,  0.6878]],
       grad_fn=<NativeBatchNormBackward0>)

同じデータが出力された。  
これは、パラメータに加え、学習データ全体の統計量も平均0, 分散1として初期化されているから。

In [10]:
norm.state_dict()

OrderedDict([('weight', tensor([1., 1., 1., 1., 1.])),
             ('bias', tensor([0., 0., 0., 0., 0.])),
             ('running_mean', tensor([0., 0., 0., 0., 0.])),
             ('running_var', tensor([1., 1., 1., 1., 1.])),
             ('num_batches_tracked', tensor(0))])

`running_mean`と`running_var`が学習データ全体の統計量。  
これらは学習モードで演算を行うと更新される。

In [11]:
norm.train()
y = norm(x)
norm.state_dict()

OrderedDict([('weight', tensor([1., 1., 1., 1., 1.])),
             ('bias', tensor([0., 0., 0., 0., 0.])),
             ('running_mean',
              tensor([-0.0288, -0.1601, -0.0387, -0.0327,  0.0778])),
             ('running_var', tensor([1.0901, 0.9263, 0.9110, 1.5726, 0.9016])),
             ('num_batches_tracked', tensor(1))])

In [12]:
print('mean:', x.mean(dim=0))
print('var:', x.var(dim=0))

mean: tensor([-0.2881, -1.6012, -0.3871, -0.3266,  0.7780])
var: tensor([1.9010, 0.2626, 0.1102, 6.7255, 0.0163])


入力したデータの平均と分散が記録された。  
`momentum=0.1`がデフォルトで設定されているので、完全には一致していない。

これでもう一度推論モードにしてみると、今記録した統計量で演算が行われる。

In [13]:
norm.eval()
y = norm(x)
y

tensor([[-1.1821, -1.8738, -0.6109,  1.2279,  0.8323],
        [ 0.6854, -1.1208, -0.1191, -1.6967,  0.6424]],
       grad_fn=<NativeBatchNormBackward0>)

確かめてみよう。

In [14]:
mean = norm.state_dict()['bias'] # 平均（パラメータ）
var = norm.state_dict()['weight'] # 分散（パラメータ）
data_mean = norm.state_dict()['running_mean'] # 平均（学習データ）
data_var = norm.state_dict()['running_var'] # 分散（学習データ）
eps = norm.eps # 微小値

((x - data_mean) / torch.sqrt(data_var + eps)) * var + mean

tensor([[-1.1821, -1.8738, -0.6109,  1.2279,  0.8323],
        [ 0.6854, -1.1208, -0.1191, -1.6967,  0.6424]])

ちゃんと同じになったね。

あと、バッチサイズ1の時もちゃんと動く。

In [15]:
x = torch.randn(1, d)
norm.eval()
norm(x)

tensor([[ 0.6924, -0.7642, -1.3633, -0.1025,  0.8151]],
       grad_fn=<NativeBatchNormBackward0>)